In [ ]:
from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
import talib
import numpy as np

# Set up Binance client
api_key = "YOUR_BINANCE_API_KEY"
api_secret = "YOUR_BINANCE_SECRET_KEY"
client = Client(api_key, api_secret)

# Constants
TRADE_SYMBOL = 'BTCFDUSD'
TRADE_QUANTITY = 0.03
BBANDS_PERIOD = 112
STD_DEV_MULTIPLIER = 2.60

# Initialize close price array
close_array = np.array([])

# Function to check BTC balance
def check_btc_balance():
    btc_balance = client.get_asset_balance(asset='BTC')['free']
    return float(btc_balance) >= TRADE_QUANTITY

holding = check_btc_balance()

# Function to execute a trade
def execute_trade(order_type, symbol, quantity):
    global holding
    try:
        print(f"Sending order {order_type} - Quantity: {quantity}")
        if order_type == 'BUY' and not holding:
            order = client.create_order(symbol=symbol, side=SIDE_BUY, type=ORDER_TYPE_MARKET, quantity=quantity)
            holding = True  # Update holding status to True after a buy
        elif order_type == 'SELL' and holding:
            order = client.create_order(symbol=symbol, side=SIDE_SELL, type=ORDER_TYPE_MARKET, quantity=quantity)
            holding = False  # Update holding status to False after a sell
        return order
    except Exception as e:
        print(f"An exception occurred - {e}")
        return None

# Calculate the Bollinger Bands and return the signal
def calculate_signal(close_array):
    if len(close_array) >= BBANDS_PERIOD:
        upperband, middleband, lowerband = talib.BBANDS(close_array, timeperiod=BBANDS_PERIOD, nbdevup=STD_DEV_MULTIPLIER, nbdevdn=STD_DEV_MULTIPLIER, matype=0)
        last_price = close_array[-1]

        if last_price < lowerband[-1]:
            return 'BUY'
        elif last_price > upperband[-1]:
            return 'SELL'
    return None

# Process incoming WebSocket messages
def process_message(msg):
    global close_array, holding
    # Extract the close price from the message
    if msg['e'] != 'error':
        candle = msg['k']
        is_candle_closed = candle['x']
        close = candle['c']

        if is_candle_closed:
            close_array = np.append(close_array, float(close))
            close_array = close_array[-BBANDS_PERIOD:]  # Keep only the latest BBANDS_PERIOD prices

            # Calculate signal
            signal = calculate_signal(close_array)

            # Update holding state
            holding = check_btc_balance()

            # Execute trade based on signal
            if signal == 'BUY' and not holding:
                execute_trade('BUY', TRADE_SYMBOL, TRADE_QUANTITY)
            elif signal == 'SELL' and holding:
                execute_trade('SELL', TRADE_SYMBOL, TRADE_QUANTITY)

# Start the WebSocket
bsm = BinanceSocketManager(client)
conn_key = bsm.start_kline_socket(TRADE_SYMBOL, process_message, interval=KLINE_INTERVAL_1MINUTE)
bsm.start()